In [1]:
# pip install sounddevice wavio speechrecognition scikit-learn


In [2]:
import tkinter as tk
from tkinter import messagebox
import sounddevice as sd
import wavio
import speech_recognition as sr
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import os
import webbrowser

In [3]:
# --- Define actions based on trigger words ---
ACTIONS = {
    "open browser": lambda: webbrowser.open("https://www.google.com"),
    "start notepad": lambda: os.system("notepad.exe")  # Windows only
}

In [4]:
# --- Training data (keywords) ---
phrases = list(ACTIONS.keys()) + [
    "hey assistant", "hello", "hi bot", "wake up", "activate",
    "what time is it", "tell me a joke", "how are you", "stop listening", "good morning"
]
labels = [1] * len(ACTIONS) + [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]  # 1 = trigger, 0 = non-trigger


In [5]:
# --- ML model ---
model = make_pipeline(TfidfVectorizer(), LogisticRegression())
model.fit(phrases, labels)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [6]:
# --- Audio functions ---
def record_audio(filename="recording.wav", duration=5, fs=44100):
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    wavio.write(filename, recording, fs, sampwidth=2)

def transcribe_audio(filename="recording.wav"):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = recognizer.record(source)
    try:
        return recognizer.recognize_google(audio)
    except:
        return ""

In [7]:
# --- Detection logic ---
def detect_trigger(text):
    if not text.strip():
        return "No speech detected."
    prediction = model.predict([text])[0]
    if prediction == 1:
        for phrase in ACTIONS:
            if phrase in text.lower():
                ACTIONS[phrase]()  # Perform action
                return f"✅ Trigger '{phrase}' detected. Action executed!"
        return "✅ Trigger word detected!"
    else:
        return "❎ Not a trigger word."

def run_detection():
    record_audio()
    text = transcribe_audio()
    result = detect_trigger(text)
    messagebox.showinfo("Result", f"You said: {text}\n\n{result}")

In [8]:
# --- GUI Setup ---
root = tk.Tk()
root.title("Trigger Word Detector with Actions")

tk.Label(root, text="🎤 Speak a command like 'open browser'...").pack(pady=10)
tk.Button(root, text="Start Listening", command=run_detection, width=30).pack(pady=10)
tk.Label(root, text="(Records for 5 seconds)").pack(pady=5)

root.mainloop()